# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse Shipments          Customer  \
0  Sep 21 2022 12:12PM  246940        10   8555202  Eywa Pharma Inc.   
1  Sep 21 2022 12:12PM  246940        10   8555205  Eywa Pharma Inc.   
2  Sep 21 2022 12:12PM  246940        10   8555206  Eywa Pharma Inc.   
3  Sep 21 2022 12:12PM  246940        10   8555209  Eywa Pharma Inc.   
4  Sep 21 2022 12:12PM  246940        10   8555210  Eywa Pharma Inc.   
5  Sep 21 2022 12:12PM  246940        10   8555211  Eywa Pharma Inc.   
6  Sep 21 2022 12:12PM  246940        10   8555212  Eywa Pharma Inc.   
7  Sep 21 2022 12:12PM  246940        10   8555213  Eywa Pharma Inc.   
8  Sep 21 2022 12:12PM  246940        10   8555215  Eywa Pharma Inc.   
9  Sep 21 2022 12:12PM  246940        10   8555216  Eywa Pharma Inc.   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
36  246937      Executing         16
37  246937       Released          8
38  246939       Released          1
39  246940       Released         19
40  246941       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
246935                NaN        NaN      20.0
246937                NaN       16.0       8.0
246939                NaN        NaN       1.0
246940                NaN        NaN      19.0
246941                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
246828               42.0       32.0       3.0
246844                8.0        8.0       0.0
246845                7.0        2.0       0.0
246864                5.0        0.0       1.0
246871                0.0        2.0       0.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
246828                 42         32         3
246844                  8          8         0
246845                  7          2         0
246864                  5          0         1
246871                  0          2         0

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               246828         42         32         3
1               246844          8          8         0
2               246845          7          2         0
3               246864          5          0         1
4               246871          0          2         0

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               246828        42        32        3
1               246844         8         8         
2               246845         7         2         
3               246864         5                  1
4               246871                   2

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                     Customer
0   Sep 21 2022 12:12PM  246940        10             Eywa Pharma Inc.
19  Sep 21 2022 12:04PM  246941        10             Eywa Pharma Inc.
20  Sep 21 2022 11:50AM  246939        12            Zelenko Labs, LLC
21  Sep 21 2022 11:36AM  246937        15  Virtus Pharmaceuticals, LLC
45  Sep 21 2022 11:34AM  246935        10             Methapharm, Inc.
65  Sep 21 2022 11:34AM  246918        15       Mizner Bioscience, LLC
69  Sep 21 2022 11:31AM  246933        19         Emersa Waterbox, LLC
70  Sep 21 2022 11:28AM  246932        10             Methapharm, Inc.
75  Sep 21 2022 10:55AM  246929        10            ISDIN Corporation
76  Sep 21 2022 10:48AM  246928        10            ISDIN Corporation

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                     Customer  \
0  Sep 21 2022 12:12PM  246940        10             Eywa Pharma Inc.   
1  Sep 21 2022 12:04PM  246941        10             Eywa Pharma Inc.   
2  Sep 21 2022 11:50AM  246939        12            Zelenko Labs, LLC   
3  Sep 21 2022 11:36AM  246937        15  Virtus Pharmaceuticals, LLC   
4  Sep 21 2022 11:34AM  246935        10             Methapharm, Inc.   
5  Sep 21 2022 11:34AM  246918        15       Mizner Bioscience, LLC   
6  Sep 21 2022 11:31AM  246933        19         Emersa Waterbox, LLC   
7  Sep 21 2022 11:28AM  246932        10             Methapharm, Inc.   
8  Sep 21 2022 10:55AM  246929        10            ISDIN Corporation   
9  Sep 21 2022 10:48AM  246928        10            ISDIN Corporation   

  Completed Executing Released  
0                           19  
1                            1  
2                            1  
3                  16        8  
4                           20  
5                   2        2  
6                            1  
7                            5  
8                            1  
9                            2

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                     Customer  \
0  Sep 21 2022 12:12PM  246940        10             Eywa Pharma Inc.   
1  Sep 21 2022 12:04PM  246941        10             Eywa Pharma Inc.   
2  Sep 21 2022 11:50AM  246939        12            Zelenko Labs, LLC   
3  Sep 21 2022 11:36AM  246937        15  Virtus Pharmaceuticals, LLC   
4  Sep 21 2022 11:34AM  246935        10             Methapharm, Inc.   
5  Sep 21 2022 11:34AM  246918        15       Mizner Bioscience, LLC   
6  Sep 21 2022 11:31AM  246933        19         Emersa Waterbox, LLC   
7  Sep 21 2022 11:28AM  246932        10             Methapharm, Inc.   
8  Sep 21 2022 10:55AM  246929        10            ISDIN Corporation   
9  Sep 21 2022 10:48AM  246928        10            ISDIN Corporation   

  Released Executing Completed  
0       19                      
1        1                      
2        1                      
3        8        16            
4       20                      
5        2         2            
6        1                      
7        5                      
8        1                      
9        2

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                     Customer  \
0  Sep 21 2022 12:12PM  246940        10             Eywa Pharma Inc.   
1  Sep 21 2022 12:04PM  246941        10             Eywa Pharma Inc.   
2  Sep 21 2022 11:50AM  246939        12            Zelenko Labs, LLC   
3  Sep 21 2022 11:36AM  246937        15  Virtus Pharmaceuticals, LLC   
4  Sep 21 2022 11:34AM  246935        10             Methapharm, Inc.   

  Released Executing Completed  
0       19                      
1        1                      
2        1                      
3        8        16            
4       20

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                     Customer  \
0  Sep 21 2022 12:12PM  246940        10             Eywa Pharma Inc.   
1  Sep 21 2022 12:04PM  246941        10             Eywa Pharma Inc.   
2  Sep 21 2022 11:50AM  246939        12            Zelenko Labs, LLC   
3  Sep 21 2022 11:36AM  246937        15  Virtus Pharmaceuticals, LLC   
4  Sep 21 2022 11:34AM  246935        10             Methapharm, Inc.   

   Released  Executing  Completed  
0      19.0        NaN        NaN  
1       1.0        NaN        NaN  
2       1.0        NaN        NaN  
3       8.0       16.0        NaN  
4      20.0        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                     Customer  \
0  Sep 21 2022 12:12PM  246940        10             Eywa Pharma Inc.   
1  Sep 21 2022 12:04PM  246941        10             Eywa Pharma Inc.   
2  Sep 21 2022 11:50AM  246939        12            Zelenko Labs, LLC   
3  Sep 21 2022 11:36AM  246937        15  Virtus Pharmaceuticals, LLC   
4  Sep 21 2022 11:34AM  246935        10             Methapharm, Inc.   

   Released  Executing  Completed  
0      19.0        0.0        0.0  
1       1.0        0.0        0.0  
2       1.0        0.0        0.0  
3       8.0       16.0        0.0  
4      20.0        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         3950617      86.0       23.0       16.0
12          493811       2.0        0.0        0.0
15          740771      32.0       27.0        0.0
16          740623       2.0       13.0        8.0
19          246933       1.0        0.0        0.0
20          740642      20.0       32.0       42.0
21          246878       1.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  3950617      86.0       23.0       16.0
1        12   493811       2.0        0.0        0.0
2        15   740771      32.0       27.0        0.0
3        16   740623       2.0       13.0        8.0
4        19   246933       1.0        0.0        0.0
5        20   740642      20.0       32.0       42.0
6        21   246878       1.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      86.0       23.0       16.0
1        12       2.0        0.0        0.0
2        15      32.0       27.0        0.0
3        16       2.0       13.0        8.0
4        19       1.0        0.0        0.0
5        20      20.0       32.0       42.0
6        21       1.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   86.0
1        12  Released    2.0
2        15  Released   32.0
3        16  Released    2.0
4        19  Released    1.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10   12    15    16   19    20   21
Status                                          
Completed  16.0  0.0   0.0   8.0  0.0  42.0  0.0
Executing  23.0  0.0  27.0  13.0  0.0  32.0  0.0
Released   86.0  2.0  32.0   2.0  1.0  20.0  1.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10   12    15    16   19    20   21
0          Completed  16.0  0.0   0.0   8.0  0.0  42.0  0.0
1          Executing  23.0  0.0  27.0  13.0  0.0  32.0  0.0
2           Released  86.0  2.0  32.0   2.0  1.0  20.0  1.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10   12    15    16   19    20   21
0  Completed  16.0  0.0   0.0   8.0  0.0  42.0  0.0
1  Executing  23.0  0.0  27.0  13.0  0.0  32.0  0.0
2   Released  86.0  2.0  32.0   2.0  1.0  20.0  1.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()